In [55]:
import os
import pandas as pd
import soundfile as sf
from tqdm import tqdm
import IPython.display as ipd
from transformers import pipeline
import librosa

In [56]:
clips_dir = "/home/morph/Desktop/FINAL/CV/cv-corpus-19.0-2024-09-13-en/cv-corpus-19.0-2024-09-13/en/clips"
validated_tsv = "/home/morph/Desktop/FINAL/CV/cv-corpus-19.0-2024-09-13-en/cv-corpus-19.0-2024-09-13/en/validated.tsv"

df_validated = pd.read_csv(validated_tsv, sep="\t")

/tmp/ipykernel_3094/784865043.py:4: DtypeWarning: Columns (4,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_validated = pd.read_csv(validated_tsv, sep="\t")


In [57]:
df_validated = df_validated.head(200_000)

entries = []

for _, row in tqdm(df_validated.iterrows(), total=len(df_validated), desc="Processing audio files"):
    audio_path = os.path.join(clips_dir, row['path'])
    sentence = row['sentence']

    with sf.SoundFile(audio_path) as f:
        duration = len(f) / f.samplerate / 60 

    entries.append({
        "audio_path": audio_path,
        "sentence": sentence,
        "duration": duration
    })

df = pd.DataFrame(entries)

Processing audio files: 100%|██████████| 200000/200000 [00:39<00:00, 5016.51it/s]


In [58]:
sum(df['duration']) / 60

311.8030066894391

In [59]:
print(df['sentence'][9998])

However, this technique also leads to the formation of several bitter and astringent flavors.


In [60]:
ipd.Audio(df['audio_path'][9998])

In [61]:
df

,audio_path,sentence,duration
0,/home/morph/Desktop/FINAL/CV/cv-corpus-19.0-20...,"Joe Keaton disapproved of films, and Buster al...",0.105125
1,/home/morph/Desktop/FINAL/CV/cv-corpus-19.0-20...,Hillel House in Leeds is home to Leeds Jewish ...,0.109925
2,/home/morph/Desktop/FINAL/CV/cv-corpus-19.0-20...,She'll be all right.,0.036469
3,/home/morph/Desktop/FINAL/CV/cv-corpus-19.0-20...,six,0.109694
4,/home/morph/Desktop/FINAL/CV/cv-corpus-19.0-20...,All's well that ends well.,0.069694
...,...,...,...
199995,/home/morph/Desktop/FINAL/CV/cv-corpus-19.0-20...,He was the last Bahraini to be released.,0.065294
199996,/home/morph/Desktop/FINAL/CV/cv-corpus-19.0-20...,Matte was an All-American back at Ohio State U...,0.087694
199997,/home/morph/Desktop/FINAL/CV/cv-corpus-19.0-20...,"The remainder of his life he devoted to study,...",0.098494
199998,/home/morph/Desktop/FINAL/CV/cv-corpus-19.0-20...,The Turtles and Splinter have different design...,0.082894


In [62]:
pipe = pipeline(model="mrrubino/wav2vec2-large-xlsr-53-l2-arctic-phoneme", device=0)

def transcribe_phonemes(audio_path):
    audio, sr = librosa.load(audio_path, sr=16000)
    return pipe(audio)["text"]

In [63]:
tqdm.pandas(desc="Generating phonemes")
df["word_phonemes"] = df["audio_path"].progress_apply(transcribe_phonemes)

Generating phonemes: 100%|██████████| 200000/200000 [1:43:44<00:00, 32.13it/s]  


In [64]:
df

,audio_path,sentence,duration,word_phonemes
0,/home/morph/Desktop/FINAL/CV/cv-corpus-19.0-20...,"Joe Keaton disapproved of films, and Buster al...",0.105125,d͡ʒoo kɪtsʌn dɪs ʌpɹuvd ʌv fʌmz ænd bʌstɚ ɔlso...
1,/home/morph/Desktop/FINAL/CV/cv-corpus-19.0-20...,Hillel House in Leeds is home to Leeds Jewish ...,0.109925,h ɛl haʊs ʌn lidz ɪz hʌm tʌ lid st͡ʒuwɪʃ sʌsaɪ...
2,/home/morph/Desktop/FINAL/CV/cv-corpus-19.0-20...,She'll be all right.,0.036469,æn sɪʌm ʃi bi ɚeɪd
3,/home/morph/Desktop/FINAL/CV/cv-corpus-19.0-20...,six,0.109694,sɪks æt
4,/home/morph/Desktop/FINAL/CV/cv-corpus-19.0-20...,All's well that ends well.,0.069694,oz wɛl ðæt ænz wɛl
...,...,...,...,...
199995,/home/morph/Desktop/FINAL/CV/cv-corpus-19.0-20...,He was the last Bahraini to be released.,0.065294,hi wʌz ðʌ læst vʌ ɪŋi tu bi ɹɪlist
199996,/home/morph/Desktop/FINAL/CV/cv-corpus-19.0-20...,Matte was an All-American back at Ohio State U...,0.087694,meɪd vʌz ʌ noʊ ʌmɛɹɪkʌn kɛk ɪ dʌ haɪoʊ steɪt j...
199997,/home/morph/Desktop/FINAL/CV/cv-corpus-19.0-20...,"The remainder of his life he devoted to study,...",0.098494,dʌ ɹɪmeɪndɚ ʌv hɪz laɪf i dʌ boʊt tu stʌdi tɹæ...
199998,/home/morph/Desktop/FINAL/CV/cv-corpus-19.0-20...,The Turtles and Splinter have different design...,0.082894,ðʌ tʌkʌtɔs ænd ʌ skɹɪntɚ hæv difɹʌn dɪzaɪn sɪn...


In [65]:
df.to_csv("/home/morph/Desktop/FINAL/CVoutput/cv_dataset.csv", index=False)
print("Dataset created successfully.")

Dataset created successfully.
